In [6]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd

In [7]:
df = pd.DataFrame(columns=['webpage_url', 'total_images'])

In [8]:
def crawl_link(link):
    try:
        link_response = BeautifulSoup(requests.get(link,
                                    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0'}).text)
        return link_response
    except Exception as e:
        return e

In [9]:
def crawl_image(link):
    try:
        link_response = requests.get(link,
                                    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0'})
        return link_response
    except Exception as e:
        return e

In [10]:
def create_folder(folder_name, parent=''):
    try:
        if parent == '':
            os.makedirs(folder_name)
        else:
            os.makedirs(f"{parent}/{folder_name}")
        return 'Folder created successfully'
    except Exception as e:
        return e

In [11]:
def extract_img(resp_data, folder_name):
    link_data = list(set([link.get('src') for link in resp_data.find_all('img')]))
    for img in link_data:
        img_name = img.split('/')[-1]
        with open(f'{os.getcwd()}/{folder_name}/{img_name}', "wb") as file:
            file.write(crawl_image(img).content)
    return len(link_data)

In [12]:
def extract_text(resp_data, folder_name):
    text_data = resp_data.text
    with open(f"{folder_name}/{folder_name}.txt", 'w') as file:
        file.write(text_data)

In [13]:
def extract_links(resp_data):
    link_data = list(set([link.get('href') for link in resp_data.find_all('a')]))    
    return link_data

In [14]:
def check_pagination(link):
    resp_data = crawl_link(link)
    pg_urls = []
    global pagi

    if pagi == False:
        try:
            total_pages = resp_data.find('div', {'class': 'wp-pagenavi'}).find('span', {'class': 'pages'}).text.split()[-1]
            for pg in range(1, int(total_pages)+1):
                pg_urls.extend(extract_links(crawl_link(f'{link}page/{pg}/')))
            pagi = True
        except Exception as e:
            print('No pagination', link)
    else:
        return []
    return pg_urls

In [20]:
pagi = False

In [16]:
landing_url = 'https://franchisesuppliernetwork.com'

In [17]:
landing_page_response=crawl_link(landing_url)

In [ ]:
links = extract_links(crawl_link('https://franchisesuppliernetwork.com/fsn-suppliers'))
for nav_link in links:
    links.extend(check_pagination(nav_link))
    if pagi:
        break
print(len(links))
pos = 0
for link in links[:1]:
    try:
        link_resp = crawl_link(link)
        folder = link[:-1].replace('https://franchisesuppliernetwork.com/', '').replace('-', '_').replace('/', '_')
        try:
            df.at[pos, 'webpage_url'] = link
            create_folder(folder)
            extract_text(link_resp, folder)
            df.at[pos, 'total_images'] =extract_img(link_resp, folder)
        except Exception as e:
            print(f"couldn't crawl this link:- {link}", e)
    except Exception as e:
        print(e, link)